In [1]:
from igraph import *
import igraph
import random
import numpy as np
import pandas as pd
from sklearn.metrics.cluster import adjusted_rand_score

path = "/Users/mcdicjh2/Desktop/gecco22/"
# Set up bipartite graph test data (should replace by proper generators e.g. from igraph package eventually)

# Define number of vertices in both levels and numerically mark as lower / upper vertex
lower = 100#100  
upper = 500#500
p_l = 0.4 # Needs to result in integer when multiplied by lower  #0.4
p_u = 0.4 # Needs to result in integer when multiplied by upper  #0.4

n_var = lower+upper
lv = np.empty(lower)
lv.fill(0)
uv = np.empty(upper)
uv.fill(1)
vertices = np.concatenate((lv, uv), axis=None)
vertices = vertices.tolist()

In [2]:
# Generate 30 different graph instances with the same connectivity stats

between_c = 10 # Specify percentage cross-community edges  #20

random.seed(20)

for it in range(0,30):

    edges = []
    connected = [0]*(lower+upper)
    #print(connected)
    for i in range(0,1000): #1000
    
        dice = random.randint(0,100)
        added = False
    
        # Find an edge that does not yet exist and add
        # Stochastically generate within community or between community edges
        while added == False:
            if dice <= 50-between_c/2:
                index1 = random.randint(0,lower*p_l-1)
                index2 = random.randint(lower,lower+upper*p_u-1)
            elif dice <= 50:
                index1 = random.randint(0,lower*p_l-1)
                index2 = random.randint(lower+upper*p_u,lower+upper-1)
            elif dice <= 50+between_c/2:
                index1 = random.randint(lower*p_l,lower-1)
                index2 = random.randint(lower,lower+upper*p_u-1)
            elif dice >= 50+between_c/2:
                index1 = random.randint(lower*p_l,lower-1)
                index2 = random.randint(lower+upper*p_u,lower+upper-1)


            newedge = [index1,index2]

            if not (newedge in edges):
                edges.append(newedge)
                connected[index1]=1
                connected[index2]=1
                added = True

   
    # Specify original ground truth 
    groundtruth=[0]*int(lower*p_l)+[1]*int(lower*(1-p_l))+[0]*int(upper*p_u)+[1]*int(upper*(1-p_u))
    shapes = ["rectangle"] * int(lower*p_l) + ["circle"] * int(lower*(1-p_l)) + ["rectangle"] * int(upper*p_u) + ["circle"] * int(upper*(1-p_u))


    # Reduce to giant component
    g_i = Graph.Bipartite(vertices, edges)
  
    index_max = np.argmax(g_i.components().sizes())

    
    #T = [groundtruth[i] for i in g_i.clusters()[index_max]]
    #vT = [vertices[i] for i in g_i.clusters()[index_max]]
    #g=g_i.clusters().giant()
    #print(g)
 
    oldelist = g_i.get_edgelist()
    oldorder= g_i.clusters()[index_max]#list(range(0,len(T)))
    order=oldorder.copy()
    #print(order)
    random.shuffle(order)
    #print(groundtruth)
    #print(vT)
    print(oldelist)
    #
    #print(oldorder)
    #print(order)
    selected = g_i.clusters()[index_max]
    
    elist=[]
    
    # Create reduced edge list
    for i in range(0,len(oldelist)):
        try :
            # Retrieve old idnex
            ti1 = oldorder.index(oldelist[i][0])
            ti2 = oldorder.index(oldelist[i][1])
            # Map vertices to new ids
            i1 = order.index(oldelist[i][0])
            i2 = order.index(oldelist[i][1])
            elist.append([i1,i2])
            #print(ti1,ti2)
            #print(i1,i2)
    
        except ValueError :
            res = "Element not in list !"
            print(res)
            

       # print(elist[i])
        #print(i1,i2)
    print(elist) 
    
   # print(order)
   # print(len(elist))
   # print(len(oldelist))
   # print(T)
   # print(vT)


    labels = [groundtruth[i] for i in order]
    topbottom = [vertices[i] for i in order]
    print(topbottom)
    #print(labels)
    #print(topbottom)
    #print(groundtruth)
    #print(vT)
    #print(len(oldelist))
    #print(len(elist))

    
    # Create actual bipartite instance
    g = Graph.Bipartite(topbottom,elist)
    #g_i = Graph.Bipartite(vT,g.get_edgelist())

    # Store instance and associated ground truth
    g.write_edgelist(path+"Graph"+str(it)+".dat")
    p = pd.DataFrame(labels)
    p.to_csv(path+"Graph"+str(it)+".truth.dat", sep=',',header=None)
    p = pd.DataFrame(topbottom)
    p.to_csv(path+"Graph"+str(it)+".vertices.dat", sep=',',header=None)






[(83, 377), (6, 183), (97, 386), (26, 204), (6, 132), (30, 248), (66, 406), (20, 260), (61, 468), (45, 260), (91, 551), (45, 152), (55, 317), (6, 121), (16, 277), (58, 456), (56, 383), (47, 311), (14, 156), (55, 328), (39, 176), (8, 277), (61, 529), (17, 242), (50, 142), (99, 280), (8, 299), (67, 476), (7, 255), (33, 149), (51, 144), (7, 175), (20, 272), (65, 534), (88, 327), (50, 360), (75, 470), (4, 246), (46, 347), (16, 264), (19, 269), (1, 149), (32, 229), (40, 380), (33, 134), (8, 207), (6, 199), (13, 161), (14, 124), (8, 242), (8, 342), (71, 301), (12, 262), (25, 124), (13, 193), (59, 373), (86, 350), (17, 115), (23, 444), (29, 350), (36, 220), (0, 279), (73, 335), (75, 474), (40, 391), (18, 255), (0, 110), (13, 258), (84, 253), (37, 192), (28, 112), (48, 492), (65, 501), (0, 299), (45, 597), (60, 417), (52, 449), (96, 472), (13, 326), (8, 123), (92, 488), (7, 222), (34, 136), (0, 135), (63, 479), (49, 309), (27, 234), (55, 500), (34, 448), (78, 451), (88, 342), (71, 437), (8, 21